<a href="https://colab.research.google.com/github/tiagoandremv/MachineLearning/blob/main/FaceUNMASKER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importação de bibliotecas

In [2]:
# IMPORTAÇÃO DE BIBLIOTECAS NECESSÁRIAS
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, mean_squared_error

from sklearn.model_selection import train_test_split

from tqdm import tqdm # permite verificar o progresso do carregamento
from random import shuffle


# usando o cv2, deixa de ser necesasrio fazer o import do google drive
#from google.colab import drive
#drive.mount('/gdrive')

class PrepareData:

  def __init__(self):
    path = '/content/drive/My Drive/Colab Notebooks/tdx_project/dataset-proj/'
    self.train_path = path + 'train/'  
    self.validation_path = path + 'validation/' 

  def label_img(self, _path):
    word_label = _path.split('/')[-2]
    if word_label == 'fake':
      return [1]
    elif word_label == 'real':
      return [0]

  def scale_image(self, img, scale_percent = 50):
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    scale = (width, height)
    img = cv2.resize(img, scale)
    return img

  def load_prepare_data (self, _type):
    dataset = []
    _path = self.train_path if _type == 'train' else self.validation_path 
    for folder in os.listdir(_path): #['fake', 'real']
      path_folder = os.path.join(_path, folder)

      for filename in tqdm(os.listdir(path_folder)):
        #print(path_folder)
        path = os.path.join(path_folder, filename)
        #print(path)
        label = self.label_img(path) # [0,1] para real , [1,0] para fake
        #print(label)

        img = cv2.imread(path)
        img = self.scale_image(img)
        img = self.scale_image(img)

        img = img.astype("float64") / 255.0
        #original_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   
        dataset.append([np.array(img), np.array(label)])
    shuffle(dataset)
    return dataset    

def plot_all_images(images):
  for i in range(len(images)):
    plt.subplot(1,len(images),i+1) 
    plt.imshow(images[i])
    plt.title(titles[i])
    plt.xticks([])
    plt.yticks([])
  plt.show()

def plot_image(img):
  plot = plt.figure()
#  plot = plot.add_subplot(4, 5, num + 1)
  plot.imshow(img)
  plot.show()





In [34]:
label_img('/content/drive/My Drive/Colab Notebooks/tdx_project/dataset/train/fake/159.jpg')

NameError: ignored

#Carregamento de Imagens

In [3]:
pData = PrepareData()
train_data = pData.load_prepare_data(_type = 'train')
validation_data = pData.load_prepare_data(_type = 'validation')

100%|██████████| 300/300 [00:24<00:00, 12.40it/s]


In [4]:
print(np.asarray(train_data).shape, np.asarray(validation_data).shape)

(1400, 2) (600, 2)


In [6]:
num, data = train_data[0]

inputs_shape = num.shape
print("Inputs Shape: ", inputs_shape)

Inputs Shape:  (256, 256, 3)


# Definição do MODEL

In [7]:
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

early_callback = EarlyStopping(monitor = 'loss', patience = 2)

model = models.Sequential()

### YOUR CODE HERE: ~9 lines of code expected, one per layer ### 
model.add(layers.Conv2D(32, (2, 2), activation='relu', padding='same', input_shape=inputs_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (2,2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(128, (2, 2), activation='relu'))
model.add(layers.Dropout(0.2))

model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10))

In [8]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy, binary_crossentropy

### YOUR CODE HERE ###
model.compile(optimizer='adam', 
              loss=SparseCategoricalCrossentropy(True), 
              metrics=['accuracy'])

Definição de features

In [9]:
# X-Features & Y-Labels
train_images = np.array([i[0] for i in train_data])
train_labels = np.array([i[1] for i in train_data]).astype('float64')

validation_images = np.array([i[0] for i in validation_data])
validation_labels = np.array([i[1] for i in validation_data]).astype('float64')

#train_images[1][1].reshape(-1, 512, 512, 1)

print(np.asarray(train_data).shape)
print(train_images.shape, train_images.dtype)
print(train_labels.shape, train_labels.dtype)

(1400, 2)
(1400, 256, 256, 3) float64
(1400, 1) float64


In [ ]:
train_labels

In [10]:
history = model.fit(train_images, 
                    train_labels, 
                    epochs = 50, 
                    validation_data = (validation_images, validation_labels), 
                    callbacks = [early_callback] )

Epoch 1/50
44/44 [==============================] - 4s 95ms/step - loss: 4.1324 - accuracy: 0.5221 - val_loss: 0.6771 - val_accuracy: 0.5950
Epoch 2/50
44/44 [==============================] - 3s 77ms/step - loss: 0.6261 - accuracy: 0.6593 - val_loss: 0.6182 - val_accuracy: 0.6567
Epoch 3/50
44/44 [==============================] - 3s 77ms/step - loss: 0.5374 - accuracy: 0.7343 - val_loss: 0.5654 - val_accuracy: 0.7050
Epoch 4/50
44/44 [==============================] - 3s 77ms/step - loss: 0.3690 - accuracy: 0.8421 - val_loss: 0.5563 - val_accuracy: 0.7233
Epoch 5/50
44/44 [==============================] - 3s 77ms/step - loss: 0.2310 - accuracy: 0.9193 - val_loss: 0.6229 - val_accuracy: 0.7333
Epoch 6/50
44/44 [==============================] - 3s 77ms/step - loss: 0.1083 - accuracy: 0.9629 - val_loss: 0.7810 - val_accuracy: 0.7367
Epoch 7/50
44/44 [==============================] - 3s 77ms/step - loss: 0.0367 - accuracy: 0.9921 - val_loss: 0.8679 - val_accuracy: 0.7417
Epoch 8/50
44

In [ ]:
train_labels

In [ ]:
# APRESENTAÇÃO DE RESULTADOS

#print("Confusion Matrix\n", confusion_matrix(y_train, train_predictions))
#print("Accuracy: %.2f %%" % (accuracy_score(y_train, train_predictions) * 100))